In [1]:
import sys
import os
home=os.getcwd()
sys.path.append(home+'/Functions')
from ChargeDataSet import *
from MS1IDs import *
from NumpyMSPeaksIdentification import *
from PeaksIdent import *
from ShowDF import *
import numpy as np
import pandas as pd
import sys
import matplotlib.pyplot as plt

In [2]:
DataSetName='12111.mzML'

In [3]:
DataSet=ChargeDataSet(DataSetName)

In [4]:
MS1IDVec=MS1IDs(DataSet)

## This one is very important!

In [163]:
ZeroInt=1e4
plus_mz=2
min_mz=200
min_mzplus=min_mz+plus_mz
max_mz=212
PeaksSafetyFactor=2
Peaks=PeaksIdent(RawSignals,minInt=ZeroInt,min_mz=0,max_mz=min_mzplus,minSignals=4,JustStats=True)
DifTres=np.max(Peaks[:,10])*PeaksSafetyFactor
Signals_filter=(RawSignals[:,0]<max_mz)&(RawSignals[:,0]>min_mz)&(RawSignals[:,1]>ZeroInt)
Signals=RawSignals[Signals_filter,:]
Dif_mz_vec=Signals[1:,0]-Signals[:-1,0]
print(np.where(Dif_mz_vec>DifTres)[0])
print(np.shape(Signals))
print(np.shape(RawSignals))

[  5  11  17  23  29  32  37  41  45  50  53  54  58  62  66  75  78  81
  84  88  94  99 101 105 108 110]
(112, 2)
(10827, 2)


In [62]:
#I can go in order and start searching for the signals between my peaks while I evaluate them
ms1_id=MS1IDVec[0]
RawSignals=np.array(DataSet[int(ms1_id)].get_peaks()).T 
Peaks=PeaksIdent(RawSignals)
for idv in np.arange(1,100,4):
    ms1_id=MS1IDVec[idv]
    RawSignals2=np.array(DataSet[int(ms1_id)].get_peaks()).T 
    NewSpectrum=RawSignals2.copy()
    DeviationTimes=4
    for peak in Peaks:
        min_mz_peak=peak[0]-DeviationTimes*peak[1]
        max_mz_peak=peak[0]+DeviationTimes*peak[1]
       # print(min_mz_peak,max_mz_peak)
        peakFilter=(NewSpectrum[:,0]>min_mz_peak)&(NewSpectrum[:,0]<max_mz_peak) 
        NewSpectrumFilter=peakFilter
        #NewSpectrum=NewSpectrum[NewSpectrumFilter,:]
        NewSpectrum[NewSpectrumFilter,:]=0
    NewPeaks=PeaksIdent(NewSpectrum)
    Peaks=np.append(Peaks,NewPeaks,axis=0)
    #print(np.shape(Peaks)[0])
    #peakLoc=np.where(peakFilter)[0]
    #print(NewSpectrum[peakLoc,0])